We will first load the data into the directory.


In [9]:
#based in  https://github.com/ynop/audiomate/blob/master/audiomate/utils/download.py


import zipfile
import requests
from collections import Counter
import sys
import os


def download_file(url, target_path=None):
    """
    Download the file from the given `url` and store it at `target_path`.
    """
    if target_path is None:
        target_path = os.path.join("data",url.split("/")[-1])

    r = requests.get(url, stream=True)

    with open(target_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    return target_path

def extract_zip(zip_path, target_folder):
    """
    Extract the content of the zip-file at `zip_path` into `target_folder`.
    """
    with zipfile.ZipFile(zip_path) as archive:
        archive.extractall(target_folder)



def get_data(fname=None, all=1):
    
    #sstopwords=set(stopwords.words('german'))
    if fname is None:        
        fname = "./data/train.txt"
        
    texts = []
    labels = []

    #split the lines into text and labels and save separately
    with open(fname) as f1:
        for l1 in f1:
            ws,lab = l1.split("\t")
            texts.append(ws)
            labels.append(lab.strip())

    #character embeddings

    #words = word_tokenize(words_raw)
    tokensents = [tk.lower().split() for tk in texts]
    words = set([word for tokens in tokensents for word in tokens])

    chars = list(' '.join(words))

    char_counts = Counter(chars)
    labels_dict = {}
    labels_nr = []
    nums = set()
    for i1,lab in enumerate(labels):
        #this step will be explained later and means, that setences with maximum of only 2 tokens should
        # be ignored
        if all==0 and len(tokensents[i1])<=2:
            continue
        nums.add(i1)
        if lab in labels_dict:
            labels_nr.append(labels_dict[lab])
        else:
            labels_dict[lab] = len(labels_dict)
            labels_nr.append(labels_dict[lab])
    tokensents = [tk for i1,tk in enumerate(tokensents) if i1 in nums]
    return labels, labels_nr, labels_dict, tokensents, words, chars, char_counts



fname="https://scholar.harvard.edu/files/malmasi/files/vardial2018-gdi-training.zip"
target_path = download_file(fname)
extract_zip( target_path, "./data/")

labels_train, labels_nr_train, labels_dict_train,sents_train_raw, words_train, chars_train, char_counts_train = get_data()
labels_dev_dev, labels_nr_dev, labels_dict_dev,sents_dev_raw, words_dev, chars_dev, char_counts_dev = get_data("./data/dev.txt")

sents_train= [" ".join(tk).lower() for tk in sents_train_raw]
sents_dev= [" ".join(tk).lower() for tk in sents_dev_raw]



So let's try to classify it.

In [10]:

from sklearn.metrics import f1_score

from sklearn.svm import LinearSVC

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

n_features=20000

clf_svm = LinearSVC(random_state=0,C=1)
tf_vectorizer = CountVectorizer( min_df=2,
                                max_features=n_features)


tf_train = tf_vectorizer.fit_transform(sents_train)

tf_dev = tf_vectorizer.transform(sents_dev)

clf_svm.fit(tf_train,labels_nr_train)

print("SVM TF weighted",f1_score(clf_svm.predict(tf_dev),labels_nr_dev, average="weighted"))
print("SVM TF macro",f1_score(clf_svm.predict(tf_dev),labels_nr_dev, average="macro"))

SVM TF weighted 0.6100404138001064
SVM TF macro 0.6036412574311785


This seems pretty already pretty good compared to last years resul, where the best was 0.66 (weighted F-1 score, https://www.aclweb.org/anthology/W/W17/W17-1201.pdf ).
But we can do much better using just a simple trick: We skip the one and two words sentences. This is because they are bad for training the classifier. Why they are bad, mostly for two reasons, too few features (only one and two words), therefore difficult to differentiate, and also very confusing labels. So same sentence being labelled with two different labels. Let's see how good this works.


In [12]:
labels_train, labels_nr_train, labels_dict_train,sents_train_raw, words_train, chars_train, char_counts_train = get_data(all=0)
labels_dev_dev, labels_nr_dev, labels_dict_dev,sents_dev_raw, words_dev, chars_dev, char_counts_dev = get_data("data/dev.txt",all=0)

sents_train= [" ".join(tk).lower() for tk in sents_train_raw]
sents_dev= [" ".join(tk).lower() for tk in sents_dev_raw]

tf_train = tf_vectorizer.fit_transform(sents_train)

tf_dev = tf_vectorizer.transform(sents_dev)

clf_svm.fit(tf_train,labels_nr_train)

print("SVM TF weighted",f1_score(clf_svm.predict(tf_dev),labels_nr_dev, average="weighted"))
print("SVM TF macro",f1_score(clf_svm.predict(tf_dev),labels_nr_dev, average="macro"))

SVM TF weighted 0.6438060442167858
SVM TF macro 0.6366767077208305


Wow, 3 points which gives already fourth place compared to last year. Surprinsingly, we are just counting the words appearing in the sentences. Normally, SVMs work best with normalized data, but that's is for another tutorial.
Let's see if there is a weighting scheme better than term frequency. Normally Term-Frequency Inverse-Document-Frequency (TF-IDF) works better for document classification.

In [13]:

tfidf_vectorizer = TfidfVectorizer( min_df=2,
                                 norm="l2",
                                   max_features=n_features)

tfidf_train = tfidf_vectorizer.fit_transform(sents_train)
tfidf_dev = tfidf_vectorizer.transform(sents_dev)

clf_svm.fit(tfidf_train,labels_nr_train)


print("SVM TF-IDF weighted",f1_score(clf_svm.predict(tfidf_dev),labels_nr_dev, average="weighted"))

print("SVM TF-IDF macro",f1_score(clf_svm.predict(tfidf_dev),labels_nr_dev, average="macro"))

SVM TF-IDF weighted 0.659225631519502
SVM TF-IDF macro 0.6514922794527852


This would made third place, almost second. But this is with SVM do others classifiers perform well?

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
import numpy as np
import scipy.sparse

clf = RandomForestClassifier(max_depth=5, n_estimators=1000, random_state=0)

clf.fit(tf_train,labels_nr_train)


print("RF TF weighted",f1_score(clf.predict(tf_dev),labels_nr_dev, average="weighted"))
print("RF TF macro",f1_score(clf.predict(tf_dev),labels_nr_dev, average="macro"))


clf.fit(tfidf_train,labels_nr_train)

print("RF TF-IDF weighted",f1_score(clf.predict(tfidf_dev),labels_nr_dev, average="weighted"))
print("RF TF-IDF macro",f1_score(clf.predict(tfidf_dev),labels_nr_dev, average="macro"))


bagging = BaggingClassifier( RandomForestClassifier(max_depth=5, n_estimators=1000, random_state=0),
                             max_samples=0.5, max_features=0.5)
bagging.fit(tf_train,labels_nr_train)


print("Bagging TF weighted",f1_score(bagging.predict(tf_dev),labels_nr_dev, average="weighted"))
print("Bagging TF macro",f1_score(bagging.predict(tf_dev),labels_nr_dev, average="macro"))
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier( RandomForestClassifier(max_depth=5, n_estimators=1000, random_state=0),
                             max_samples=0.5, max_features=0.5)
bagging.fit(tfidf_train,labels_nr_train)


print("Bagging TFIDF weighted",f1_score(bagging.predict(tfidf_dev),labels_nr_dev, average="weighted"))
print("Bagging TFIDF macro",f1_score(bagging.predict(tfidf_dev),labels_nr_dev, average="macro"))

bagging = BaggingClassifier( RandomForestClassifier(max_depth=5, n_estimators=1000, random_state=0),
                             max_samples=0.5, max_features=0.5)

bagging.fit(scipy.sparse.hstack([tf_train,tfidf_train]).tocsr(),labels_nr_train)


print("Bagging TF+TFIDF weighted",f1_score(bagging.predict(scipy.sparse.hstack([tf_dev,tfidf_dev]).tocsr()),labels_nr_dev, average="weighted"))
print("Bagging TF+TFIDF macro",f1_score(bagging.predict(scipy.sparse.hstack([tf_dev,tfidf_dev]).tocsr()),labels_nr_dev, average="macro"))


RF TF weighted 0.4885197964113094
RF TF macro 0.4221673549708851
RF TF-IDF weighted 0.4893916483789577
RF TF-IDF macro 0.4228223976071996
Bagging TF weighted 0.46183188110927015
Bagging TF macro 0.3985855877443282
Bagging TFIDF weighted 0.5021817720653874
Bagging TFIDF macro 0.3917232063144712
Bagging TF+TFIDF weighted 0.47146645251840025
Bagging TF+TFIDF macro 0.39716687771209946


Bagging and RF sometimes are good sometimes are bad, also using SVM decreases bagging the performance.
We now try with n-ngrams of chars

In [15]:
# additionally use normlized tfidf char bigrams 
tf_vectorizer_char_ngram = CountVectorizer( min_df=2,
                                               analyzer="char",
                                   max_features=n_features, ngram_range=(1, 7))
tf_train_char_ngram = tf_vectorizer_char_ngram.fit_transform(sents_train)
tf_dev_char_ngram = tf_vectorizer_char_ngram.transform(sents_dev)

clf_svm.fit(tf_train_char_ngram,labels_nr_train)
preds=clf_svm.predict(tf_dev_char_ngram)
print("SVM TF-IDF tf char ngram 7",f1_score(preds,labels_nr_dev, average="weighted"))
print("SVM TF-IDF tf char ngram macro",f1_score(preds,labels_nr_dev, average="macro"))

SVM TF-IDF tf char ngram 7 0.6558271945540242
SVM TF-IDF tf char ngram macro 0.6468589036370523


In [16]:

# additionally use normlized tfidf char bigrams 
tfidf_vectorizer_char_ngram = TfidfVectorizer( min_df=2,
                                 norm="l2",
                                               analyzer="char",
                                   max_features=n_features, ngram_range=(1, 7))
tfidf_train_char_ngram = tfidf_vectorizer_char_ngram.fit_transform(sents_train)
tfidf_dev_char_ngram = tfidf_vectorizer_char_ngram.transform(sents_dev)

clf_svm.fit(tfidf_train_char_ngram,labels_nr_train)
preds=clf_svm.predict(tfidf_dev_char_ngram)
print("SVM TF-IDF tf char ngram 7",f1_score(preds,labels_nr_dev, average="weighted"))
print("SVM TF-IDF tf char ngram macro",f1_score(preds,labels_nr_dev, average="macro"))



SVM TF-IDF tf char ngram 7 0.6774906696759666
SVM TF-IDF tf char ngram macro 0.6658621351916394


Wow, now we would win the competitions from 2017. Here we see also tfidf works well also for sentences and chars. But this was out of the box, can we do better bigrams? Why bigrams, because we are looking to features which are  phonetically similar and similarly structured, not quite perfect match. Also the number of aas and ee might give good hints. I'll show you why the tfidf_vectorizer with char n-gram could do even better:

In [17]:
tfidf_vectorizer_char_ngram.get_feature_names()[:20]

[' ',
 ' a',
 ' a ',
 ' a d',
 ' a d ',
 ' a de',
 ' a de ',
 ' aa',
 ' aa ',
 ' aab',
 ' aabe',
 ' aaf',
 ' aafa',
 ' aafan',
 ' aafang',
 ' aag',
 ' aagf',
 ' aagfa',
 ' aagfan',
 ' aagl']

We can see lots of white spaces, which do not provide a good insight on the structure of the words. Although, silence is a important part of the music.


In [18]:
def gather_bigrams(data):
    res = set()
    for n1 in data:
        res.update(bigrams(n1))
    return list(res)

def bigrams(word):
    chars = [c for c in word]
    bigrams = [c1 + c2 for c1, c2 in zip(chars, chars[1:])]
    features = chars + bigrams
    return features

def transform_features(data_train, data_test, n_grams=1):

    bigrams_list = gather_bigrams([tj for tk in data_train for tj in tk.split() if tj.find(" ")==-1])
    cv = CountVectorizer(
            analyzer=bigrams,
       # analyzer="char",
            preprocessor=lambda x : x,
            vocabulary=bigrams_list,
        ngram_range=(1, n_grams))

    
    X_train = cv.fit_transform(data_train)

    X_test = cv.transform(data_test)
    return X_train, X_test

X_train, X_test = transform_features(sents_train,sents_dev)

clf_svm.fit(X_train,labels_nr_train)
preds=clf_svm.predict(X_test)
print("SVM CV + Bigrams 4",f1_score(preds,labels_nr_dev, average="weighted"))
print("SVM CV + Bigrams macro",f1_score(preds,labels_nr_dev, average="macro"))
clf_svm.fit(scipy.sparse.hstack([tfidf_train,X_train]),labels_nr_train)
preds=clf_svm.predict(scipy.sparse.hstack([tfidf_dev.todense(),X_test]))
print("SVM CV + Bigrams 4",f1_score(preds,labels_nr_dev, average="weighted"))
print("SVM CV + Bigrams macro",f1_score(preds,labels_nr_dev, average="macro"))

SVM CV + Bigrams 4 0.5910983725307404
SVM CV + Bigrams macro 0.5789372643322429
SVM CV + Bigrams 4 0.6773374236772861
SVM CV + Bigrams macro 0.6700263168459002


Although alone these simple bigram analyzer performs poorly in combination with tf-idf, it increases the macro f-1 considerably, 0.005 is what differentiate mostly between the places.
What is the impact of using ngram instead of words for tf-idf (we keep the char bigrams).

In [19]:
tfidf_vectorizer_ngram = TfidfVectorizer( min_df=2,
                                 norm="l2",
                                   max_features=n_features, ngram_range=(1, 7))
tfidf_train_ngram = tfidf_vectorizer_ngram.fit_transform(sents_train)
tfidf_dev_ngram = tfidf_vectorizer_ngram.transform(sents_dev)
 
clf_svm.fit(scipy.sparse.hstack([tfidf_train_ngram,X_train]),labels_nr_train)
preds=clf_svm.predict(scipy.sparse.hstack([tfidf_dev_ngram,X_test]))
print("SVM TF-IDF + Bigrams + word 7 ngram 5",f1_score(preds,labels_nr_dev, average="weighted"))
print("SVM TF-IDF + Bigrams + word 7 ngram macro",f1_score(preds,labels_nr_dev, average="macro"))

SVM TF-IDF + Bigrams + word 7 ngram 5 0.678517820542825
SVM TF-IDF + Bigrams + word 7 ngram macro 0.6686880145927554


So less macro and more weighted.

In [20]:
X_train_ngrams, X_test_ngrams = transform_features(sents_train,sents_dev, n_grams=7)

clf_svm.fit(scipy.sparse.hstack([tfidf_train_ngram,X_train_ngrams]),labels_nr_train)
preds=clf_svm.predict(scipy.sparse.hstack([tfidf_dev_ngram,X_test_ngrams]))
print("SVM TF-IDF + Bigrams  + char 7 ngram 6",f1_score(preds,labels_nr_dev, average="weighted"))
print("SVM TF-IDF + Bigrams  + char 7 ngram macro",f1_score(preds,labels_nr_dev, average="macro"))


SVM TF-IDF + Bigrams  + char 7 ngram 6 0.678517820542825
SVM TF-IDF + Bigrams  + char 7 ngram macro 0.6686880145927554


So no changes. Let's try to put even more features together.

In [21]:
clf_svm.fit(scipy.sparse.hstack([tfidf_train_ngram,X_train_ngrams,tfidf_train_char_ngram]),labels_nr_train)
preds=clf_svm.predict(scipy.sparse.hstack([tfidf_dev_ngram,X_test_ngrams,tfidf_dev_char_ngram]))
print("SVM TF-IDF + Bigrams  + char 7 ngram (bigrams) tfidf chars ngrams + tf char ngram 7",f1_score(preds,labels_nr_dev, average="weighted"))
print("SVM TF-IDF + Bigrams  + char 7 ngram (bigrams) tfidf chars ngrams + tf char ngram macro",f1_score(preds,labels_nr_dev, average="macro"))


SVM TF-IDF + Bigrams  + char 7 ngram (bigrams) tfidf chars ngrams + tf char ngram 7 0.6807815919337346
SVM TF-IDF + Bigrams  + char 7 ngram (bigrams) tfidf chars ngrams + tf char ngram macro 0.6708978004436262


Nice, more for both features.
So lets try normalizing the tf and using char ngram count

In [22]:
from sklearn.preprocessing import normalize

tf_vectorizer_char_ngram = CountVectorizer( min_df=2,
                                               analyzer="char",
                                   max_features=n_features, ngram_range=(1, 7))
tf_train_char_ngram = tf_vectorizer_char_ngram.fit_transform(sents_train)
tf_dev_char_ngram = tf_vectorizer_char_ngram.transform(sents_dev)

tf_train_char_ngram = normalize(tf_train_char_ngram)
tf_dev_char_ngram = normalize(tf_dev_char_ngram)


clf_svm.fit(scipy.sparse.hstack([tfidf_train_ngram,X_train_ngrams,tfidf_train_char_ngram,tf_train_char_ngram]),labels_nr_train)
preds=clf_svm.predict(scipy.sparse.hstack([tfidf_dev_ngram,X_test_ngrams,tfidf_dev_char_ngram,tf_dev_char_ngram]))
print("SVM TF-IDF + Bigrams  + char 7 ngram (bigrams) tfidf chars ngrams + tf char ngram 8",f1_score(preds,labels_nr_dev, average="weighted"))
print("SVM TF-IDF + Bigrams  + char 7 ngram (bigrams) tfidf chars ngrams + tf char ngram macro",f1_score(preds,labels_nr_dev, average="macro"))


SVM TF-IDF + Bigrams  + char 7 ngram (bigrams) tfidf chars ngrams + tf char ngram 8 0.6817307419317699
SVM TF-IDF + Bigrams  + char 7 ngram (bigrams) tfidf chars ngrams + tf char ngram macro 0.6725325917465754


Ok, not much difference but still increased.
Until now we just concatenated the features, but they are quite different, and can cause the SVMs to give more important to heavier features. A possible counter action would be to normalize it. Yet, since we have many different features and many different possible ways to normalize (over train, over feature over sample), we will use separate svms and use voting (ensemble voting).

In [23]:

svms=[]
for tk in [tfidf_train_ngram,X_train_ngrams,tfidf_train_char_ngram,tf_train_char_ngram]:
    svms.append(LinearSVC(random_state=0, C=1))
    svms[-1].fit(tk,labels_nr_train)

svms_preds=[]
for i1,tk in enumerate([tfidf_dev_ngram,X_test_ngrams,tfidf_dev_char_ngram,tf_dev_char_ngram]):
    svms_preds.append(svms[i1].predict(tk))


    
#sumpres=sum([tk for tk in svms_preds])
sumpres=[Counter(np.array(svms_preds)[:,tk]).most_common()[0][0] for tk in range(len(svms_preds[0]))]
print("SVM TF-IDF separate svm for features Bigrams   char 7 ngram 9",f1_score(sumpres,labels_nr_dev, average="weighted"))
print("SVM TF-IDF separate svm for features Bigrams   char 7 ngram macro",f1_score(sumpres,labels_nr_dev, average="macro"))


SVM TF-IDF separate svm for features Bigrams   char 7 ngram 9 0.6894441553644064
SVM TF-IDF separate svm for features Bigrams   char 7 ngram macro 0.6758401343396775


Cool, not bad, almost one point in weighted, but again, the vote is not weighted. So we count each vote equal, although the classifier might be not so convinced of the prediction.


In [24]:
svms=[]
for tk in [tf_train, tfidf_train, tfidf_train_ngram,X_train_ngrams,tfidf_train_char_ngram,tf_train_char_ngram]:
    svms.append(LinearSVC(random_state=0, C=1))
    svms[-1].fit(tk,labels_nr_train)

svms_preds=[]
for i1,tk in enumerate([tf_dev, tfidf_dev,tfidf_dev_ngram,X_test_ngrams,tfidf_dev_char_ngram,tf_dev_char_ngram]):
    svms_preds.append(svms[i1].decision_function(tk))
    
sumpres=sum([tk for tk in svms_preds])
print("SVM TF-IDF separate svm for features Bigrams   char 7 ngram 10",f1_score(np.argmax(sumpres,1),labels_nr_dev, average="weighted"))
print("SVM TF-IDF separate svm for features Bigrams   char 7 ngram macro",f1_score(np.argmax(sumpres,1),labels_nr_dev, average="macro"))


SVM TF-IDF separate svm for features Bigrams   char 7 ngram 10 0.6970817681702751
SVM TF-IDF separate svm for features Bigrams   char 7 ngram macro 0.687829000954753


Much better. almost 0.70, we would have beaten nicely the first place of last year.
Still, a sum seems poor... can be build a meta classifier?

In [25]:
svms=[]
for tk in [tfidf_train_ngram,X_train_ngrams,tfidf_train_char_ngram,tf_train_char_ngram]:
    svms.append(LinearSVC(random_state=0, C=1))
    svms[-1].fit(tk,labels_nr_train)

svms_train_meta=[]
for i1,tk in enumerate([tfidf_train_ngram,X_train_ngrams,tfidf_train_char_ngram,tf_train_char_ngram]):
    svms_train_meta.append(svms[i1].decision_function(tk))

svm_meta=LinearSVC(random_state=0, C=0.75)
svm_meta.fit(scipy.sparse.hstack([np.concatenate(svms_train_meta,1),tfidf_train_ngram,X_train_ngrams,tfidf_train_char_ngram,tf_train_char_ngram]),labels_nr_train)



svms_preds=[]
for i1,tk in enumerate([tfidf_dev_ngram,X_test_ngrams,tfidf_dev_char_ngram,tf_dev_char_ngram]):
    svms_preds.append(svms[i1].decision_function(tk))
svm_meta_preds=svm_meta.decision_function(scipy.sparse.hstack([np.concatenate(svms_preds,1),tfidf_dev_ngram,X_test_ngrams,tfidf_dev_char_ngram,tf_dev_char_ngram]))


print("Meta SVM TF-IDF separate svm for features Bigrams   char 7 ngram",f1_score(np.argmax(svm_meta_preds,1),labels_nr_dev, average="macro"))



Meta SVM TF-IDF separate svm for features Bigrams   char 7 ngram 0.6678147793590107


That was not good, probably the classifiers did too well in the training set. The solution is to do a cross validation, i.e. slice the training data e.g. in 5 slices, use one slice for test and the others for training, and iterate, switching the test slice every time (5 times in total). We make predictions on the test slice, gathering all predictions creates a prediction set which we can train a meta classifier. I did not refactor the code, so it can be quite long to go through but that is the main idea. We use the helper function perform from meta_cv. Let's see the results:

In [50]:
imp.reload(meta_cv)
import meta_cv
from meta_cv import MetaCV

#download gold labels
gold_dir="./data/gold/"
if not os.path.exists(gold_dir):
    os.makedirs(gold_dir)
target_path_gold=download_file("https://drive.google.com/uc?authuser=0&id=0B8I6bgWbt_MfWHJrNDhpOFVVcUhabGFxWDJUWG9LR2hGODFJ&export=download", target_path=gold_dir+"testdata_fixed.zip")
extract_zip( target_path_gold, gold_dir)

def perform(comp):
    meta=meta_cv.MetaCV(splits=5)
    print ("comp",comp)
    labels_train, labels_nr_train, labels_dict_train,sents_train_raw, words_train, chars_train, char_counts_train = get_data()
    labels_dev_dev, labels_nr_dev, labels_dict_dev,sents_dev_raw, words_dev, chars_dev, char_counts_dev = get_data(fname="./data/dev.txt")
    sents_train= [" ".join(tk).lower() for tk in sents_train_raw]
    sents_dev= [" ".join(tk).lower() for tk in sents_dev_raw]
    labels_dev_dev, labels_nr_dev, labels_dict_dev,sents_dev_raw, words_dev, chars_dev, char_counts_dev = get_data(fname="./data/dev.txt")
        
    labels_test, labels_nr_test, labels_dict_test,sents_test_raw, words_test, chars_test, char_counts_test = get_data(fname="./data/gold/gold.txt")

    if comp==0:
        meta.fit(sents_train,labels_nr_train)
        preds=meta.predict(sents_dev)
        print("SVM TF",f1_score(preds,labels_nr_dev, average="weighted"))
        print("SVM TF macro",f1_score(preds,labels_nr_dev, average="macro"))
    else:
        rev_labels_dict_train = dict([(tv,tk) for tk,tv in labels_dict_train.items()])
        sents_train_comp= [" ".join(tk).lower() for tj in [sents_train_raw,sents_dev_raw] for tk in tj]
        labels_nr_train_comp = [ tk for tj in [labels_nr_train,labels_nr_dev] for tk in tj]
        meta.fit(sents_train_comp,labels_nr_train_comp) 
        sents_test_comp= [" ".join(tk).lower() for tk in sents_test_raw]
        preds=meta.predict(sents_test_comp)
        labels_preds=[rev_labels_dict_train[np.argmax(tk)] for tk in preds]
        with open("prediction.labels","w") as f1:
            for label in labels_preds:
                f1.write(label+"\n")

        from sklearn.model_selection import cross_val_score

        preds_score=meta.decision_function(sents_test_comp)

        trf=np.argmax(preds_score,1)

        rev_labels_dict_train=dict([(tv,tk) for tk,tv in labels_dict_train.items()])
        rev_labels_dict_train[-1]="XY"

        tr_labs=[rev_labels_dict_train[trf[tk]] for tk in range(trf.shape[0])] 

        with open("predictions_c5_metacv_multiclass_threshold.labels","w") as f1:
            for tr1 in tr_labs:
                f1.write(tr1+"\n")

        gdi4=np.isin(labels_nr_test,[labels_dict_test[tk] for tk in ["ZH","LU","BE","BS"]])
        index=np.where(gdi4)[0]

        print("SVM TF",f1_score([labels_dict_test[rev_labels_dict_train[tk]] for tk in np.array(trf)[index]],np.array(labels_nr_test)[index], average="weighted"))
        print("SVM TF macro",f1_score([labels_dict_test[rev_labels_dict_train[tk]] for tk in np.array(trf)[index]],np.array(labels_nr_test)[index], average="macro"))

        return index, gdi4,trf, labels_dict_test,rev_labels_dict_train,labels_nr_test, preds_score

perform(0)

comp 0
SVM TF 0.666387461900244
SVM TF macro 0.6564319851674072


Nice, now we are set to compete.

In [45]:
import bigrams
meta_cv.perform(0,1,0)

uall 0 comp 1 tf 0


NameError: name 'get_data' is not defined

 With only one feature, the winner did win with multiple features, and they were quite good also before. Can we mix them somehow and get better results? 

In [ ]:
imp.reload(meta_cv)